In [13]:
# 설치하기
#!pip install selenium

In [9]:
import selenium
print(selenium.__version__)

4.25.0


## Selenium 모듈 사용
- BeautifulSoup 모듈과 비슷하게 HTML 문서의 구조에 따라 접근할 수 있음
- BeautifulSoup 에서 원하는 태그, 속성등을 가져오기 위해 단일 객체는 find(), 복수 객체는 find_all() 함수로 접근했다면 selenium에서는 태그 종류에 따라 각각 함수가 있음


## Chromedriver 다운로드하기
- https://chromedriver.chromium.org/downloads : https://storage.googleapis.com/chrome-for-testing-public/123.0.6312.58/win64/chromedriver-win64.zip

## chromedriver 직접 다운로드 하지 않고 진행하려면 다음 패키지를 설치하기
- !pip install webdriver_manager

## selenium 사용해보기
### stpe1 : chromedriver 서비스 사용하여 브라우저 시작하기

In [14]:
# stpe1 : chromedriver 서비스 사용하여 브라우저 시작하기

import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# chromedriver.exe 경로 지정하기
# path = os.path.join(os.getcwd(), "chromedriver.exe")
# 서비스를 생성하고 chromedriver의 경로를 지정합니다..
# service = webdriver.chrome.service.Service(path)
# 지정된 서비스를 사용하여 Chrome 브라우저를 시작합니다.
# driver = webdriver.Chrome(service=service)

chrome_options = Options()
# Selenium이 브라우저를 실제로 열지 않고 작동하게 하려면 옵션을 설정합니다.
#chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않는 옵션입니다.

# 웹드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

### step2 : 지정한 웹페이지에 접속하기

In [15]:
# webdriver가 google 페이지에 접속하도록 명령
driver.get("http://www.google.co.kr")

# 페이지 제목을 확인하여 Google 사이트에 접속했는지 확인하기.
assert "Google" in driver.title

### step3 : 검색 진행하기

In [16]:
# 검색 입력 부분에 커서를 올리고
#검색 입력 부분에 다양한 명령을 내리기 위해 elem 변수에 할당
elem = driver.find_element("name", "q")

#입력 부분에 지운다-default로 값이 있을 수 있는 경우 고려
elem.clear()
 
#검색어 입력
elem.send_keys("Selenium")
 
#검색 실행
elem.submit()
 
#검색이 제대로 됐는지 확인
assert "No results found." not in driver.page_source

### step4 : 종료하기

In [17]:
# webdriver를 종료하여 창이 사라진다
driver.close()

### 네이버 금융에서 최근 인기종목 5개 수집

In [18]:
import os
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

def get_stock_info():

    # Selenium이 브라우저를 실제로 열지 않고 작동하게 하려면 옵션을 설정합니다.
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않는 옵션입니다.

    # 웹드라이버 초기화
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    url = "https://finance.naver.com/sise/lastsearch2.nhn"
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'lxml')

    #data_rows = soup.find("table", attrs={"class":"type_5"}).find("tbody").find_all("tr")
    data_rows = soup.find("table", class_ ="type_5").find("tbody").find_all("tr")
    #print(data_rows)
    
    for idx, row in enumerate(data_rows):
        columns = row.find_all("td")
        #print(columns)
        if len(columns) <= 1:  # 의미없는 데이터 스킵
            continue
        data = [column.get_text().strip() for column in columns]
        print(f"종목명: {data[1]:10s}, 현재가: {data[3]:>7s}, 전일대비: {data[4]:>5s}")
        
        if idx >= 6:  # 상위 5개만 출력
            break
    
    time.sleep(1)  
    driver.quit()

get_stock_info()

종목명: 삼성전자      , 현재가:  59,000, 전일대비: 하락
				200
종목명: 유한양행      , 현재가: 160,000, 전일대비: 상승
				4,700
종목명: 루미르       , 현재가:  14,910, 전일대비: 상승
				2,910
종목명: SK하이닉스    , 현재가: 190,900, 전일대비: 상승
				3,600
종목명: 인스피언      , 현재가:  20,550, 전일대비: 상한가
				4,730


## Selenium+BeautifulSoup 응용 - 폼 양식 전송 실습
- selenium + Web driver 를 활용하여 버튼을 클릭하거나 자바스크립트 삽입 등 특정 조건이 충족되어야만 접근 가능한 데이터를 모듈을 활용하여 가져올 수 있음

### Step1 : selenium을 활용한 naver 로그인 및 메일 탭 클릭
- 활성 탭 전환하기 : https://devyurim.github.io/python/crawler/2018/08/13/crawler-3.html

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import os

chrome_options = Options()
#chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않는 옵션입니다.

# 웹드라이버 초기화
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

url = "https://www.naver.com"
driver.get(url)

element = driver.find_element(By.CLASS_NAME,'MyView-module__link_login___HpHMW')

element.click()

In [21]:
# **** 부분에 자신의 네이버 ID/PWD 입력하여 실행하세요

id = '*******'
pw = '*******'


In [22]:
# 로그인 시 자동입력방지하기 위한 captcha 프로그램 실행을 방지하기 위해 직접 아이디와 패스워드값을 프로그램에 입력하는 것이 좋음
driver.execute_script("document.getElementById('id').value=\'" + id + "\'")
# element = driver.find_element(By.ID, "id")
# element.send_keys(id)
time.sleep(1)

# element = driver.find_element(By.ID, "pw")
# element.send_keys(pw)

driver.execute_script("document.getElementById('pw').value=\'" + pw + "\'")
time.sleep(1)

element = driver.find_element(By.CLASS_NAME,'btn_text')
element.click()

time.sleep(1)
element = driver.find_element(By.CLASS_NAME,'service_icon.type_mail')
element.click()

In [8]:
# 최근 열린 탭으로 전환
driver.switch_to.window(driver.window_handles[-1])

# 로딩 기다리기
time.sleep(5)

### Step2 : 메일함 페이지 소스 가져오기

In [23]:
# 'soup'는 웹 페이지의 BeautifulSoup 객체라고 가정합니다.

page_source = driver.page_source
soup = BeautifulSoup(page_source, 'html.parser')
# 이메일 목록의 아이템을 모두 찾습니다.
# 모든 메일 항목을 가져옵니다.
mail_items = soup.find_all('div', class_='mail', limit=10)

for item in mail_items:
    # 'button_sender' 클래스를 가진 버튼 내부의 텍스트에서 '보낸 사람' 정보를 추출합니다.
    sender_button = item.find('button', class_='button_sender')
    if sender_button:
        blind_span = sender_button.find('span', class_='blind')
        if blind_span:
            blind_span.extract()  # 'blind' 클래스를 가진 span을 제거합니다.
        sender_name = sender_button.get_text(strip=True)  # 남은 텍스트를 추출합니다.
    
    # 'mail_title' 클래스를 가진 a 태그에서 '메일 제목'을 추출합니다.
    title_link = item.find('a', class_='mail_title_link')
    mail_title = title_link.get_text(strip=True) if title_link else "Title Not Found"
    
    print(f"보낸 사람: {sender_name}, 메일 제목: {mail_title}")

보낸 사람: 네이버, 메일 제목: 메일 제목새로운 환경에서 로그인 되었습니다.
보낸 사람: 지피터스 GPTers, 메일 제목: 메일 제목송다혜 (닿) sent you a message on 지피터스 GPTers
보낸 사람: Adobe, 메일 제목: 메일 제목인증 코드
보낸 사람: Adobe, 메일 제목: 메일 제목인증 코드
보낸 사람: 지피터스 뉴스레터, 메일 제목: 메일 제목#101. ⏰ D-Day | 지피터스 AI캠프가 오늘 마감됩니다.
보낸 사람: KB국민은행, 메일 제목: 메일 제목(KB국민은행-계약관리안내) 입출금이자유로운예금약관 개정 안내
보낸 사람: 지피터스 뉴스레터, 메일 제목: 메일 제목#100. AI 활용 사례, 이것만 보세요.
보낸 사람: 삼성생명, 메일 제목: 메일 제목[삼성생명] 강*숙 고객님 2024-03-20일 보험료 자동이체 결과 안내드립니다.
보낸 사람: 지피터스 뉴스레터, 메일 제목: 메일 제목#99. 달리3, 프롬프트 단축 버튼 추가
보낸 사람: 모두의연구소, 메일 제목: 메일 제목(광고)자, 이제 시작이야 내 꿈을~ 내 꿈을 위한 아이펠


### **연습문제 1**
* 방식1 : selenium 사용하여 네이버 현재 날씨 상태 정보 가져오기 
* url - https://weather.naver.com/

In [ ]:
# 1. 패키지 가져오고 URL을 요청한다.
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

# 2.브라우저를 헤드리스 모드로 설정합니다.

# Code here


# 3. WebDriver를 설정합니다.


# 4.웹 페이지를 가져옵니다.


# 5.webdriver가 url 페이지에 접속하도록 명령
# Code here


# WebDriverWait을 사용하여 요소가 나타날 때까지 기다립니다.
# wait = WebDriverWait(driver, 10)


try:
    # '현재온도'를 가져옵니다.
    

    # '날씨 상태'를 가져옵니다.
   

    # 정보를 출력합니다.
    
    
except Exception as e:
    print(f"An error occurred: {e}")

# WebDriver 세션을 종료합니다.
driver.quit()


현재 온도
18.5°, 흐림
어제보다 2.2°
높아요


### **연습문제 2**
* 방식2 : selenium + beautiful 사용하여 네이버 현재 날씨 상태 정보 가져오기 
* url - https://weather.naver.com/

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup  # BeautifulSoup 임포트
import time  # 간단한 지연을 위해 사용

# 브라우저를 헤드리스 모드로 설정합니다.


# WebDriver를 설정합니다.


# 웹 페이지를 가져옵니다.


# 페이지 로딩을 위한 잠깐의 지연
time.sleep(3)  # 필요에 따라 3초 이상으로 조정 가능

try:
    # 페이지의 전체 HTML을 가져옵니다.
    
    # BeautifulSoup을 사용하여 HTML 파싱합니다.
    

    # '현재온도'를 찾습니다.
    current_temp = 

    # '날씨 상태'를 찾습니다.
    weather_status = 

    # 정보를 출력합니다.
    

except Exception as e:
    print(f"An error occurred: {e}")

# WebDriver 세션을 종료합니다.
driver.quit()

현재 온도18.5°, 흐림어제보다2.2°높아요
